In [1]:
import pandas as pd
from sklearn import datasets

iris = datasets.load_iris()
X = iris.data
y = iris.target
feature_names = iris.feature_names
target_names = iris.target_names

# Création d'un DataFrame pandas
df = pd.DataFrame(X, columns=feature_names)
df['species'] = pd.Categorical.from_codes(y, target_names)

# Affichage des premières lignes du DataFrame
print(df.head(150))

     sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                  5.1               3.5                1.4               0.2   
1                  4.9               3.0                1.4               0.2   
2                  4.7               3.2                1.3               0.2   
3                  4.6               3.1                1.5               0.2   
4                  5.0               3.6                1.4               0.2   
..                 ...               ...                ...               ...   
145                6.7               3.0                5.2               2.3   
146                6.3               2.5                5.0               1.9   
147                6.5               3.0                5.2               2.0   
148                6.2               3.4                5.4               2.3   
149                5.9               3.0                5.1               1.8   

       species  
0       se

In [1]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split

# Charger le dataset IRIS
iris = datasets.load_iris()
X, y = iris.data, iris.target
classes = np.unique(y)

# Séparer le dataset en données d'entraînement et de test en utilisant la stratification
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Afficher les formes des données pour vérifier la division
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

# Calculer les probabilités a priori pour chaque classe
def calculate_prior(y_train, classes):
    priors = {}
    total_count = len(y_train)
    class_counts = np.bincount(y_train)  # Compte le nombre d'instances pour chaque classe
    for c in classes:
        priors[c] = class_counts[c] / total_count  # Calcule la probabilité a priori pour chaque classe
    return priors

# Afficher les probabilités a priori
priors = calculate_prior(y_train, classes)
print("Probabilités a priori pour chaque classe :")
for cls, prob in priors.items():
    print(f"Classe {cls}: {prob:.4f}")

# Calculer les moyennes et variances pour chaque attribut par classe
def calculate_mean_variance(X_train, y_train, classes):
    mean_var = {}
    for c in classes:
        X_c = X_train[y_train == c]  # Sélectionne les instances de la classe c
        mean_var[c] = {
            'mean': np.mean(X_c, axis=0),  # Calcule la moyenne des attributs pour la classe c
            'var': np.var(X_c, axis=0)  # Calcule la variance des attributs pour la classe c
        }
    return mean_var

# Affiche les moyennes et variances des attributs par classe
mean_var = calculate_mean_variance(X_train, y_train, classes)
print("\nMoyennes et variances pour chaque attribut par classe :")
for cls, stats in mean_var.items():
    print(f"Classe {cls}:")
    print(f"  Moyennes: {stats['mean']}")
    print(f"  Variances: {stats['var']}")

# Densité de probabilité d'une distribution normale
def gaussian_density(x, mean, var):
    exponent = np.exp(-((x - mean) ** 2) / (2 * var))  # Calcule la partie exponentielle de la densité
    return (1 / np.sqrt(2 * np.pi * var)) * exponent  # Calcule la densité de probabilité

# Calcul de la probabilité postérieure pour une instance donnée
def calculate_posterior(X, priors, mean_var, classes):
    posteriors = []
    for c in classes:
        prior = np.log(priors[c])  # Prend le logarithme de la probabilité a priori
        conditional = np.sum(np.log(gaussian_density(X, mean_var[c]['mean'], mean_var[c]['var'])))  # Calcule la somme des logs des vraisemblances
        posterior = prior + conditional  # Calcule la log-probabilité postérieure
        posteriors.append(posterior)
    return classes[np.argmax(posteriors)], posteriors  # Retourne la classe avec la plus grande probabilité postérieure et les probabilités

# Entraînement du modèle
priors = calculate_prior(y_train, classes)
mean_var = calculate_mean_variance(X_train, y_train, classes)

# Prédiction de la classe pour une nouvelle instance
def predict(X, priors, mean_var, classes):
    predictions = []
    for instance in X:
        predicted_class, posteriors = calculate_posterior(instance, priors, mean_var, classes)
        predictions.append(predicted_class)
    return np.array(predictions)

# Prédictions sur le dataset de test
y_pred = predict(X_test, priors, mean_var, classes)

# Évaluation de la précision
accuracy = np.mean(y_pred == y_test)
print(f'\nAccuracy: {accuracy * 100:.2f}%')

# Saisie d'une nouvelle instance
print("\nEntrez les valeurs pour une nouvelle instance :")
new_instance = np.array([
    float(input("Longueur du sépale (4.3 - 7.9 cm) : ")),
    float(input("Largeur du sépale (2.0 - 4.4 cm) : ")),
    float(input("Longueur du pétale (1.0 - 6.9 cm) : ")),
    float(input("Largeur du pétale (0.1 - 2.5 cm) : "))
]).reshape(1, -1)

# Affichage des valeurs de la nouvelle instance
print("\nValeurs de la nouvelle instance :")
print(f"Longueur du sépale: {new_instance[0, 0]} cm")
print(f"Largeur du sépale: {new_instance[0, 1]} cm")
print(f"Longueur du pétale: {new_instance[0, 2]} cm")
print(f"Largeur du pétale: {new_instance[0, 3]} cm")
# Prédire la classe pour la nouvelle instance
predicted_class, posteriors = calculate_posterior(new_instance, priors, mean_var, classes)
print(f'\nClasse prédite pour la nouvelle instance: {predicted_class}')

# Affichage des log-probabilités postérieures pour la nouvelle instance
print("\nLog-probabilités postérieures pour la nouvelle instance :")
for cls, posterior in zip(classes, posteriors):
    print(f"Classe {cls}: {posterior:.4f}")

# Affichage de la densité de probabilité pour la nouvelle instance
print("\nDensités de probabilité pour la nouvelle instance :")
for cls, stats in mean_var.items():
    densities = gaussian_density(new_instance, stats['mean'], stats['var'])
    print(f"Classe {cls}: {densities}")


X_train shape: (105, 4)
X_test shape: (45, 4)
y_train shape: (105,)
y_test shape: (45,)
Probabilités a priori pour chaque classe :
Classe 0: 0.3333
Classe 1: 0.3333
Classe 2: 0.3333

Moyennes et variances pour chaque attribut par classe :
Classe 0:
  Moyennes: [4.98857143 3.42571429 1.48571429 0.24      ]
  Variances: [0.10329796 0.1739102  0.02293878 0.00925714]
Classe 1:
  Moyennes: [5.94857143 2.73142857 4.23714286 1.30857143]
  Variances: [0.24078367 0.08558367 0.21147755 0.03564082]
Classe 2:
  Moyennes: [6.68285714 3.00857143 5.63142857 2.06857143]
  Variances: [0.42484898 0.1173551  0.32272653 0.06386939]

Accuracy: 91.11%

Entrez les valeurs pour une nouvelle instance :

Valeurs de la nouvelle instance :
Longueur du sépale: 5.0 cm
Largeur du sépale: 3.0 cm
Longueur du pétale: 2.0 cm
Largeur du pétale: 2.0 cm

Classe prédite pour la nouvelle instance: 1

Log-probabilités postérieures pour la nouvelle instance :
Classe 0: -172.1315
Classe 1: -21.2190
Classe 2: -25.1353

Densités 